In [ ]:
import numpy as np
# from sage.plot.plot3d.shapes import surface

# Domain resolution
Nx, Ny = 300, 150
x_vals = np.linspace(-np.pi, np.pi, Nx)
y_vals = np.linspace(-2, 2, Ny)

# Mesh grid
X, Y = np.meshgrid(x_vals, y_vals)

# Vectorized evaluation: sin(x + i y)
Z = np.sin(X) * np.cosh(Y) + 1j * np.cos(X) * np.sinh(Y)

# Magnitude as height, phase as color
mag = np.abs(Z)
phase = (np.angle(Z) + np.pi) / (2*np.pi)  # Normalize to [0,1]

# Build surface manually from grid
# surface takes lists-of-lists for heights and colors
mag_list = mag.tolist()
phase_list = phase.tolist()

surf = surface(list(zip(mag_list, phase_list)), X, Y)
surf.show(aspect_ratio=[1,1,0.4], frame=True, colorbar=True)

In [ ]:
from sage.plot.plot3d.plot3d import plot3d_adaptive
x, y = var('x,y')
p = plot3d_adaptive(sin(x*y), (x, -pi, pi), (y, -pi, pi), initial_depth=5)

p.show()

In [ ]:
x,y,z = var('x y z')
plot_vector_field3d((cos(x), cos(y), cos(z)),(x,0,4 * pi), (y,0,4 * pi), (z,0,4 * pi), plot_points=10, arrowsize=100, arrowwidth=100)
plot_vector_field((sin(x), sin(y)),(x,0,4 * pi), (y,0,4 * pi), plot_points=40)

In [ ]:
var('x y')
f = (x + I*y)^2

# Separate real and imaginary parts
u = real_part(f)
v = imag_part(f)

# Plot vector field
N = 2
plot_vector_field((u, v), (x, -N, N), (y, -N, N))

In [ ]:
RANGE = 10
ARROWS = RANGE * 5
POINTS = RANGE * 100

def scale_log(z_out):
    z_arg = z_out.abs()
    return z_out * (log(1 + z_arg) / z_arg)

scale_abs = lambda z_val: scale_log(scale_log(z_val))

##

var('x y', domain=RR)
var('z', domain=CC)
z_xy = (x + y * I)
f = cos(z) #z^2

# Calculate its derivative
df = diff(f, z)

# From 1-ary complex to 2-ary real as a trivial orthogonal basis
f_xy = f(z=z_xy)
df_xy = scale_abs(df(z=z_xy))

# Real and imaginary parts of the derivative (for arrows)
du = real_part(df_xy)
dv = imag_part(df_xy)

# Colored background for the value f(z)
from sage.plot.complex_plot import complex_plot
color_field = complex_plot(f, (x, -RANGE, RANGE), (y, -RANGE, RANGE), plot_points=POINTS) # colorbar=True, cmap='hsv', alpha=0.6

# Plot arrows for derivative f'(z)
from sage.plot.plot_field import plot_vector_field
vec_field = plot_vector_field((du, dv), (x, -RANGE, RANGE), (y, -RANGE, RANGE), plot_points=ARROWS,
                              headwidth=3, headlength=5, pivot='middle') # arrowsize=2, arrowwidth=1

# Combine and show
(color_field + vec_field).show(figsize=10)

In [ ]:
RANGE = 10
ARROWS = RANGE * 5
POINTS = RANGE * 100

def scale_log(z_out):
    z_arg = z_out.abs()
    return z_out * (log(1 + z_arg) / z_arg)

scale_abs = lambda z_val: scale_log(scale_log(z_val))

##
from sage.plot.complex_plot import complex_plot
from sage.plot.plot_field import plot_vector_field

var('x y', domain=RR)
var('z', domain=CC)
z_xy = (x + y * I)

def plot_complex_c2dvec(
    z_func,
    zxy_map=z_xy,

    axis_range=(-RANGE, RANGE),
    num_arrows=ARROWS,
    num_points=POINTS,

    scale_len=scale_abs,
):
    # Calculate its derivative
    dz_func = diff(z_func, z)
    # From 1-ary complex to 2-ary real as a trivial orthogonal basis
    dxy_func = dz_func(z=z_xy)
    # Scale appropriately to account for crazy arrow differences
    arrow_func = scale_len(dxy_func)

    # Real and imaginary parts of the derivative (for arrows)
    dxy_u = real_part(arrow_func)
    dxy_v = imag_part(arrow_func)

    # Plot arrows for derivative f'(z)
    vec_field = plot_vector_field(
        (dxy_u, dxy_v),
        (x, *axis_range), (y, *axis_range),
        plot_points=num_arrows,
        headwidth=3,
        headlength=4, headaxislength=4,
        pivot='middle'
    ) # color=???

    # Colored background for the value f(z)
    color_field = complex_plot(
        z_func,
        (x, *axis_range), (y, *axis_range),
        plot_points=num_points
    ) # colorbar=True, cmap='hsv', alpha=0.6

    return color_field + vec_field


plot_complex_c2dvec(
    z_func=cos(z)
).show(figsize=10)